In [1]:
import requests
import time

# Define variables
ticker = 'bitcoin'
currency = 'usd'
api_key = 'CG-WRMmzZqKfjsyK2AKfd9YkUzR'

# URL and headers
url = f"https://api.coingecko.com/api/v3/simple/price?ids={ticker}&vs_currencies={currency}"
headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": api_key
}

# Retrieve prices in a loop with error handling
try:
    while True:
        response = requests.get(url, headers=headers)
        
        # Check if the response was successful
        if response.status_code == 200:
            data = response.json()
            last_price = data[ticker][currency]
            print(f"Price of {ticker} in {currency}: {last_price}")
        else:
            print("Failed to retrieve data:", response.status_code, response.text)
        
        # Wait 60 seconds before the next request
        time.sleep(60)

except KeyboardInterrupt:
    print("Program interrupted by user.")
except Exception as e:
    print(f"An error occurred: {e}")


Price of bitcoin in usd: 67578
Program interrupted by user.


In [1]:
import pandas as pd
import requests
from datetime import datetime

def fetch_crypto_data(crypto_pair, start_date, api_key):
    # Parse the crypto_pair
    coin, currency = crypto_pair.split('/')
    
    # Convert start_date to a datetime object and calculate the number of days since start_date
    start_date_obj = datetime.strptime(start_date, "%Y-%m-%d")
    days = (datetime.now() - start_date_obj).days
    
    # CryptoCompare endpoint for historical daily data
    url = f"https://min-api.cryptocompare.com/data/v2/histoday"
    
    # Define parameters for the API request
    params = {
        'fsym': coin.upper(),
        'tsym': currency.upper(),
        'limit': days,  # Number of days since start_date
        'toTs': int(datetime.now().timestamp()),  # To current date
        'api_key': api_key
    }
    
    # Fetch data from CryptoCompare API
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        raise Exception(f"Error fetching data: {response.status_code}, {response.text}")
    
    # Parse JSON data
    data = response.json()['Data']['Data']
    
    # Convert the data to a DataFrame
    df = pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['time'], unit='s')
    
    # Select required columns and rename
    df = df[['Date', 'open', 'high', 'low', 'close']]
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close']
    
    # Filter data from the specified start_date
    df = df[df['Date'] >= start_date]
    
    return df

# Example usage (replace 'YOUR_API_KEY' with your actual CryptoCompare API key)
api_key = "123d0d4d92c39b8718008f333cc712003f1ab8150795e6e1bfd5b6259350ba1b"
df = fetch_crypto_data("BTC/USD", "2023-01-01", api_key)
print(df)

          Date      Open      High       Low     Close
0   2023-01-01  16531.31  16619.82  16502.85  16613.87
1   2023-01-02  16613.87  16768.07  16544.15  16670.08
2   2023-01-03  16670.08  16766.70  16605.01  16670.16
3   2023-01-04  16670.16  16972.62  16651.02  16846.82
4   2023-01-05  16846.82  16869.84  16764.64  16825.87
..         ...       ...       ...       ...       ...
668 2024-10-30  72723.67  72945.06  71408.40  72331.96
669 2024-10-31  72331.96  72675.24  69585.02  70207.54
670 2024-11-01  70207.54  71616.01  68755.75  69473.41
671 2024-11-02  69473.41  69884.38  69006.88  69354.01
672 2024-11-03  69354.01  69364.80  67845.35  68548.40

[673 rows x 5 columns]
